In [56]:
import pandas as pd
import torch
from constants import ALL_COLS, LABEL

In [81]:
model = torch.nn.Sequential(
    torch.nn.Linear(len(ALL_COLS), 64), 
    torch.nn.ReLU(),
    
    torch.nn.Linear(64, 32),
    torch.nn.ReLU(),
    
    torch.nn.Linear(32, 2)   
)

In [82]:
opt = torch.optim.Adam(model.parameters(), lr=0.0005)
m = torch.nn.LogSoftmax(dim=1)
nll_loss = torch.nn.NLLLoss()

raw_df = pd.read_csv('data_raw/train.csv')
raw_train = torch.tensor(raw_df[ALL_COLS].values, dtype=torch.float32)
raw_label = torch.tensor(raw_df[LABEL].values, dtype=torch.long)


In [83]:
for _ in range(15):
    loss = nll_loss(m(model(raw_train)), raw_label)

    opt.zero_grad()
    loss.backward()
    opt.step()

    print(loss)

tensor(12.5955, grad_fn=<NllLossBackward0>)
tensor(11.1313, grad_fn=<NllLossBackward0>)
tensor(9.7192, grad_fn=<NllLossBackward0>)
tensor(8.3170, grad_fn=<NllLossBackward0>)
tensor(6.9246, grad_fn=<NllLossBackward0>)
tensor(5.5421, grad_fn=<NllLossBackward0>)
tensor(4.1688, grad_fn=<NllLossBackward0>)
tensor(2.8582, grad_fn=<NllLossBackward0>)
tensor(1.7695, grad_fn=<NllLossBackward0>)
tensor(0.8420, grad_fn=<NllLossBackward0>)
tensor(0.8805, grad_fn=<NllLossBackward0>)
tensor(1.8212, grad_fn=<NllLossBackward0>)
tensor(2.2522, grad_fn=<NllLossBackward0>)
tensor(2.1764, grad_fn=<NllLossBackward0>)
tensor(1.7487, grad_fn=<NllLossBackward0>)
